In [ ]:
#from __future__ import absolut_import, division, print_function

# Import Tensorflow and Tensorflow Datasets
import tensorflow as tf
import tensorflow_datasets as tfds
tf.logging.set_verbosity(tf.logging.ERROR)

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

# Improve progress bar display
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

print(tf.__version__)

# enable eager execution if we are below Tensorflow 2
tf.enable_eager_execution()

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: <class '_frozen_importlib._ModuleLockManager'> returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
num_train_samples = metadata.splits['train'].num_examples
num_test_samples = metadata.splits['test'].num_examples
print("Number of training examples:\t\t{}".format(num_train_samples))
print("Number of test examples:\t\t{}".format(num_test_samples))

# Preprocessing

Let's first transform the images to contain values from 0-1 instead of 0-255. We do this with a normalize function:

In [ ]:
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

# map applies the normalize function element-wise
train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

In [ ]:
# take a single image and remove color dimension by reshaping
for image, label in test_dataset.take(1):
    break
image = image.numpy().reshape((28, 28))

# plot image to show what it contains
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(test_dataset.take(25)):
    image = image.numpy().reshape((28,28))
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])
plt.show()

# Build the model

So let's start by building the model up in keras and then compiling it.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

Now we have a model with three layers:

1. *input* layer called `tf.keras.layers.Flatten` which transforms the original images from 28x28 pixels to a 1D-array of 784 pixels. Here nothing is learned, we simplay transform our input

2. *hidden* layer called `tf.keras.layers.Dense` with 128 neurons and a ReLU activation function. It takes the 784 pixels as input values and weights them according to weights it will learn during training. It then outputs 128 values, 1 for each neuron.

3. *output* layer, again a `tf.keras.layers.Dense`. This time it has 10 neurons which represent the 10 different output classes. The softmax activation function will transform the outputs to 10 probabilities in the range `[0, 1]` resembling the probabilities for each class.

# Compile the model

Before we start training we need to *compile* the model. In this step we need to add a few other things:

* *Loss function*: it determines how far away our output is from the ground truth. This is what we want to minimize during training. Here we decide to take the `sparse_categorical_crossentropy`

* *Optimizer*: This algorithm adjusts the weights inside the layers to minimize the loss. We take the `adam` optimizer.

* *Metrics*: This monitors the training and testing steps. A simple metric is `accuracy` as it simply counts the fraction of the correctly classified images.

In [ ]:
model.compile(optimizer=tf.adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])